In [1]:
%load_ext autoreload
%autoreload 2
import TRITON_SWMM_toolkit.examples as examples
import TRITON_SWMM_toolkit.analysis as analysis
from TRITON_SWMM_toolkit.process_simulation import TRITONSWMM_sim_post_processing
import TRITON_SWMM_toolkit.process_simulation as ps
import TRITON_SWMM_toolkit.processing_analysis as pe
from TRITON_SWMM_toolkit.process_simulation import *
import TRITON_SWMM_toolkit.plot_utils as plot_utils
import TRITON_SWMM_toolkit.utils as utils 
import TRITON_SWMM_toolkit.run_simulation as run
import TRITON_SWMM_toolkit.sensitivity_analysis as snst
from TRITON_SWMM_toolkit.config import analysis_config
from TRITON_SWMM_toolkit.examples import GetTS_TestCases as tst
from TRITON_SWMM_toolkit.scenario import TRITONSWMM_scenario
from importlib import reload
reload(examples)
reload(analysis)
reload(ps)
reload(pe)
reload(plot_utils)
reload(utils)
reload(snst)
rebuild_case = False

# single sim test case

In [5]:
testcase = tst.retrieve_norfolk_single_sim_test_case(
        start_from_scratch=False, download_if_exists=False
    )


In [6]:
testcase.system.process_system_level_inputs()

In [7]:
testcase.system.compile_TRITON_SWMM(redownload_triton_swmm_if_exists=False, recompile_if_already_done_successfully=False)

TRITON-SWMM already compiled


# multi-sim testcase

In [11]:
testcase = tst.retrieve_norfolk_multi_sim_test_case(start_from_scratch=False)
analysis = testcase.system.analysis

In [13]:
# analysis = norfolk_multi_sim_analysis

result = analysis.submit_workflow(
    mode="local",
    process_system_level_inputs=True,
    overwrite_system_inputs=True,
    compile_TRITON_SWMM=True,
    recompile_if_already_done_successfully=True,
    prepare_scenarios=True,
    overwrite_scenario=True,
    rerun_swmm_hydro_if_outputs_exist=True,
    process_timeseries=True,
    which="both",
    clear_raw_outputs=True,
    overwrite_if_exist=True,
    compression_level=5,
    pickup_where_leftoff=False,
    dry_run=False,
    verbose=True,
)

[Snakemake] Submitting workflow in local mode
[Snakemake] Snakefile generated: /home/dcl3nd/dev/TRITON-SWMM_toolkit/test_data/norfolk_coastal_flooding/tests/multi_sim/Snakefile
[Snakemake] Running workflow locally with Snakemake
[Snakemake] DRY RUN
[Snakemake] Using dynamic config from: /home/dcl3nd/dev/TRITON-SWMM_toolkit/test_data/norfolk_coastal_flooding/tests/multi_sim/.snakemake_profile/local


[Snakemake] Workflow completed successfully
[Snakemake] Running workflow locally with Snakemake
[Snakemake] Using dynamic config from: /home/dcl3nd/dev/TRITON-SWMM_toolkit/test_data/norfolk_coastal_flooding/tests/multi_sim/.snakemake_profile/local
[Snakemake] Workflow completed successfully


In [9]:
analysis.SWMM_node_summary

ValueError: could not open file because it does not exist: /home/dcl3nd/dev/TRITON-SWMM_toolkit/test_data/norfolk_coastal_flooding/tests/multi_sim/SWMM_nodes.zarr. Run analysis.consolidate_[SWMM/TRITON]_outputs().

# sensitivity analysis

In [4]:
nrflk_cpu_sensitivity = tst.retrieve_norfolk_cpu_config_sensitivity_case(
        start_from_scratch=True
    )
analysis = nrflk_cpu_sensitivity.system.analysis

In [6]:
result = analysis.submit_workflow(
    mode="local",
    process_system_level_inputs=True,
    overwrite_system_inputs=True,
    compile_TRITON_SWMM=True,
    recompile_if_already_done_successfully=True,
    prepare_scenarios=True,
    overwrite_scenario=True,
    rerun_swmm_hydro_if_outputs_exist=True,
    process_timeseries=True,
    which="both",
    clear_raw_outputs=True,
    overwrite_if_exist=True,
    compression_level=5,
    pickup_where_leftoff=False,
    dry_run=True,
    verbose=True,
)

[Snakemake] Submitting sensitivity analysis workflow in local mode
[Snakemake] Generated master Snakefile: /home/dcl3nd/dev/TRITON-SWMM_toolkit/test_data/norfolk_coastal_flooding/tests/cpu_config_sensitivity/Snakefile
[Snakemake] Created required directories (_status, logs/sims)
[Snakemake] Running workflow locally with Snakemake
[Snakemake] Using dynamic config from: /home/dcl3nd/dev/TRITON-SWMM_toolkit/test_data/norfolk_coastal_flooding/tests/cpu_config_sensitivity/.snakemake_profile/local
[Snakemake] Workflow completed successfully


In [ ]:
testcase = tst.retreive_norfolk_cpu_config_sensitivity_case(start_from_scratch=False)
analysis = testcase.system.analysis

In [ ]:
analysis.sensitivity.compile_TRITON_SWMM_for_sensitivity_analysis()
assert analysis.sensitivity.compilation_successful == True

In [ ]:
analysis.sensitivity.prepare_scenarios_in_each_subanalysis(concurrent=True)
assert analysis.log.all_scenarios_created.get() == True

In [ ]:
analysis.sensitivity.run_all_sims(pickup_where_leftoff=False, concurrent=False)
# assert analysis.log.all_sims_run.get() == True